In [1]:
from google.colab import drive
drive.mount("/content/drive")
!pip install optuna

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 12.8 MB/s eta 0:00:00


In [16]:
import os
import cv2
import zipfile
import numpy as np
import tensorflow as tf
import gc

from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    TimeDistributed, LSTM, Bidirectional
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [23]:
SEQ_LEN = 16
IMG_SIZE = 112
EPOCHS = 20

WORK_DIR = "/content/drive/MyDrive/WORK_CCD"

ACC_ZIP = "/content/drive/MyDrive/CarCrash/CarCrash/videos/Crash-1500.zip"
NOR_ZIP = "/content/drive/MyDrive/CarCrash/CarCrash/videos/Normal.zip"

ACC_DIR = os.path.join(WORK_DIR, "crash")
NOR_DIR = os.path.join(WORK_DIR, "normal")


In [24]:
def unzip_if_needed(zip_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to, exist_ok=True)
        with zipfile.ZipFile(zip_path, "r") as z:
            z.extractall(extract_to)
        print(f"Extracted {zip_path}")


In [25]:
def collect_video_paths():
    paths, labels = [], []

    for v in os.listdir(ACC_DIR):
        paths.append(os.path.join(ACC_DIR, v))
        labels.append(1)

    for v in os.listdir(NOR_DIR):
        paths.append(os.path.join(NOR_DIR, v))
        labels.append(0)

    return paths, labels


video_paths, labels = collect_video_paths()
print("Total videos:", len(video_paths))


Total videos: 4500


In [26]:
from sklearn.model_selection import train_test_split

train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    video_paths, labels,
    test_size=0.3,
    stratify=labels,
    random_state=42
)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels,
    test_size=0.5,
    stratify=temp_labels,
    random_state=42
)

In [27]:
def train_sequence_generator(video_paths, labels):
    for path, label in zip(video_paths, labels):
        cap = cv2.VideoCapture(path)
        frames = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = frame.astype("float32") / 255.0
            frames.append(frame)

            if len(frames) == SEQ_LEN:
                yield np.array(frames), label
                frames = []   # non-overlapping

        cap.release()


In [28]:
def single_sequence_generator(video_paths, labels):
    for path, label in zip(video_paths, labels):
        cap = cv2.VideoCapture(path)
        frames = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = frame.astype("float32") / 255.0
            frames.append(frame)

            if len(frames) == SEQ_LEN:
                yield np.array(frames), label
                break   # 🚨 STOP after first sequence

        cap.release()


In [29]:
def make_train_dataset(paths, labels, batch_size):
    ds = tf.data.Dataset.from_generator(
        lambda: train_sequence_generator(paths, labels),
        output_signature=(
            tf.TensorSpec((SEQ_LEN, IMG_SIZE, IMG_SIZE, 3), tf.float32),
            tf.TensorSpec((), tf.int32)
        )
    )
    ds = ds.shuffle(128).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


def make_eval_dataset(paths, labels, batch_size):
    ds = tf.data.Dataset.from_generator(
        lambda: single_sequence_generator(paths, labels),
        output_signature=(
            tf.TensorSpec((SEQ_LEN, IMG_SIZE, IMG_SIZE, 3), tf.float32),
            tf.TensorSpec((), tf.int32)
        )
    )
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [30]:
def build_model():
    inputs = tf.keras.Input(shape=(SEQ_LEN, IMG_SIZE, IMG_SIZE, 3))

    x = TimeDistributed(Conv2D(16, 3, activation="relu"))(inputs)
    x = TimeDistributed(MaxPooling2D())(x)

    x = TimeDistributed(Conv2D(32, 3, activation="relu"))(x)
    x = TimeDistributed(MaxPooling2D())(x)

    x = TimeDistributed(Flatten())(x)
    x = Dropout(0.4)(x)

    x = LSTM(128)(x)
    x = Dropout(0.4)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(3e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [31]:
train_ds = make_train_dataset(train_paths, train_labels, batch_size=4)
val_ds = make_eval_dataset(val_paths, val_labels, batch_size=4)
test_ds = make_eval_dataset(test_paths, test_labels, batch_size=4)

model = build_model()

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    steps_per_epoch=100,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    ]
)


Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 440s 4s/step - accuracy: 0.6993 - loss: 0.5532 - val_accuracy: 0.7733 - val_loss: 0.4691
Epoch 2/20


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


100/100 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.8219 - loss: 0.3900 - val_accuracy: 0.8267 - val_loss: 0.3726
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.8334 - loss: 0.3685 - val_accuracy: 0.8741 - val_loss: 0.3431
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.8729 - loss: 0.2897 - val_accuracy: 0.7659 - val_loss: 0.4364
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.8927 - loss: 0.2186 - val_accuracy: 0.8756 - val_loss: 0.3079
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.8582 - loss: 0.3037 - val_accuracy: 0.8370 - val_loss: 0.3448
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.8726 - loss: 0.3097 - val_accuracy: 0.9007 - val_loss: 0.2432
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.9138 - loss: 0.1994 - val_accuracy: 0.9037 - val_loss: 0.2520
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.9573 - loss: 0.1485 - val_accuracy: 0.893

In [32]:
loss, acc = model.evaluate(test_ds)
print(f"TEST ACCURACY (video-level): {acc:.4f}")

169/169 ━━━━━━━━━━━━━━━━━━━━ 309s 2s/step - accuracy: 0.9314 - loss: 0.1957
TEST ACCURACY (video-level): 0.9259
